In [4]:
!pip install datascience
!pip install wget
!pip install plotly

In [5]:
from datascience import *
from plotly.offline import plot
import numpy as np
import wget
import plotly.plotly as py
import plotly.graph_objs as go

In [14]:
csvUrl = 'https://raw.githubusercontent.com/SivanC/Data-Science/master/Homework%20Graphing/HW%20Data%20-%20Sheet3.csv'
homeworkDataCsv = wget.download(csvUrl)

In [57]:
#With my local version of the data
homeworkData = Table().read_table(r'C:\Users\Sivan Cooperman\Desktop\python\Data-Science\Homework Graphing\exampleData.csv').relabeled('HW Time (min)', 'Homework').relabeled('Other Work Time (min)', 'Other').relabeled('Total (min)', 'Total')

In [58]:
def generatePoints(dates):
    """Takes an array of dates and retrieves the time spent doing homework, other work, and total work on those days.
    Ex:
    >>> generatePoints(['1/1', '2/5'])
        [['1/1', '2/5'], [31, 26], [4, 180], [37, 206]]
    """
    hwMinutes = []
    otherMinutes = []
    totalMinutes = []
    
    for date in dates:
        hwMinutes.append(homeworkData.where('Date', date)['Homework'][0])
        otherMinutes.append(homeworkData.where('Date', date)['Other'][0])
        totalMinutes.append(homeworkData.where('Date', date)['Total'][0])
        
    return [dates, hwMinutes, otherMinutes, totalMinutes]

In [143]:
def graphForDates(dates, dataType='subtotals', notes=False):
    """Takes an array of dates and plots their associated minutes of work
        Arguments:
            dataType: all, total, subtotals, homework, or other. Determines the type of data displayed on the graph.
            notes: whether or not to show annotations from the notes column of homeworkData
    """
    
    hwMinutes, otherMinutes, totalMinutes = generatePoints(dates)[1:]
    
    annotations = []
    if notes:
        for note in homeworkData['Notes']:
            if note != 'nan':
                newDict = dict(x=homeworkData.where('Notes', note)['Date'][0],
                               y=homeworkData.where('Notes', note)['Total'][0],
                               xref='x',
                               yref='y',
                               align='left',
                               width=120,
                               font=dict(color='black'),
                               visible=False,
                               clicktoshow='onoff',
                               xclick=dates[1],
                               yclick=totalMinutes[1] + 50,
                               text=note)
                annotations.append(newDict)
    
    hwBar = go.Bar(x=dates, y=hwMinutes, text=hwMinutes, hoverinfo='text', name = 'Homework')
    otherBar = go.Bar(x=dates, y=otherMinutes, text=otherMinutes, hoverinfo='text', name = 'Other')
    totalBar = go.Bar(x=dates, y=totalMinutes, text=totalMinutes, hoverinfo='text', name = 'Total')
    annotationPoint = go.Scatter(x=[dates[1]], y=[totalMinutes[1]+50],
                                 showlegend=False, name='Show Notes', hoverinfo='name')

    if dataType == 'all':
        title = 'Minutes of Work'
        data = [totalBar, otherBar, hwBar, annotationPoint]
    elif dataType == 'total':
        title = 'Total Minutes of Work'
        data = [totalBar]
    elif dataType == 'subtotals':
        title = 'Minutes of Homework and Other Work'
        data = [otherBar, hwBar]
    elif dataType == 'homework':
        title = 'Minutes of Homework'
        data = [hwBar]
    elif dataType == 'other':
        title = 'Minutes of Other Work'
        data = [otherBar]
    else:
        raise ValueError('dataType must be one of the following: all, total, subtotals, homework, other')
        
    layout = dict(title=title,
                  hovermode='closest',
                  barmode='relative',
                  xaxis=dict(
                      title='Date'),
                  yaxis=dict(
                      title='Minutes'),
                  annotations=annotations
    )
    
    figure = dict(data=data, layout=layout)
    plot(figure, filename='graphForDates' + dates[0].replace('/', '') + 'to' + dates[-1].replace('/', '') + '.html')

In [145]:
graphForDates(homeworkData['Date'], notes=True)